In [ ]:
import json
import os
import requests
import tweepy
import openai
import asyncio
from datetime import datetime
import pytz
import nest_asyncio
import time
import random
import platform
import hmac
import hashlib
from openai import OpenAI

# APIキーとトークンの読み込み
config_file = 'twitter_key_tokou_use.json'
if not os.path.exists(config_file):
    raise FileNotFoundError(f"{config_file} が存在しません。")

with open(config_file, 'r') as f:
    try:
        config = json.load(f)
        print("Config file loaded successfully:")
        print(json.dumps(config, indent=4))
    except json.JSONDecodeError as e:
        raise ValueError(f"JSONファイルの読み込みに失敗しました: {e}")

# 必要なキーが存在するか確認
required_keys = ['rakuten', 'twitter', 'openai']
for key in required_keys:
    if key not in config:
        raise KeyError(f"必要なキー '{key}' が {config_file} に存在しません。")

rakuten_app_id = config['rakuten']['application_id']
rakuten_affiliate_id = config['rakuten']['affiliate_id']
rakuten_app_secret = config['rakuten']['application_secret']
twitter_consumer_key = config['twitter']['consumer_key']
twitter_consumer_secret = config['twitter']['consumer_secret']
twitter_bearer_token = config['twitter']['bearer_token']
twitter_access_token = config['twitter']['access_token']
twitter_access_token_secret = config['twitter']['access_token_secret']
openai_api_key = config['openai']['api_key']

# APIキーを使用してクライアントを設定
client = OpenAI(api_key=openai_api_key)

# システム情報の取得
def get_system_info():
    system = platform.system()
    node = platform.node()
    release = platform.release()
    version = platform.version()
    machine = platform.machine()
    processor = platform.processor()

    return {
        "System": system,
        "Node Name": node,
        "Release": release,
        "Version": version,
        "Machine": machine,
        "Processor": processor
    }

# 楽天ランキングの取得
def get_ranking():
    api_url = 'https://app.rakuten.co.jp/services/api/IchibaItem/Ranking/20170628'
    params = {
        'applicationId': rakuten_app_id,
        'affiliateId': rakuten_affiliate_id,
        'format': 'json',
        'genreId': 0  # 全ジャンル
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()

    # デバッグのためにレスポンスを表示
    print("API Response:")
    print(json.dumps(data, indent=4))
    
    items = []
    if 'Items' in data:
        for rank in data['Items'][:3]:  # 上から3つ取得
            item = {}
            item_data = rank['Item']
            item['title'] = item_data['itemName']
            item['url'] = item_data['itemUrl']
            item['image'] = item_data['mediumImageUrls'][0]['imageUrl']
            item['item_id'] = item_data['itemCode']
            item['details'] = item_data.get('catchcopy', '詳細情報が見つかりませんでした。')
            items.append(item)
    else:
        print("Error: 'Items' not found in response")
    
    return items

# サムネイル画像の保存
def save_images(items):
    if not os.path.exists('images'):
        os.makedirs('images')
    
    for i, item in enumerate(items):
        response = requests.get(item['image'])
        with open(f'images/item_{i+1}.jpg', 'wb') as f:
            f.write(response.content)

# 楽天アフィリエイトリンクの取得（署名付き）
def get_affiliate_links(items, application_id, affiliate_id, app_secret):
    api_url = 'https://app.rakuten.co.jp/services/api/IchibaItem/Search/20170706'
    
    for item in items:
        params = {
            'applicationId': application_id,
            'affiliateId': affiliate_id,
            'keyword': item['title'],
            'format': 'json'
        }
        query_string = '&'.join([f'{k}={v}' for k, v in sorted(params.items())])
        signature = hmac.new(app_secret.encode(), query_string.encode(), hashlib.sha256).hexdigest()
        params['signature'] = signature
        
        response = requests.get(api_url, params=params)
        data = response.json()
        
        if 'Items' in data:
            item['affiliate_link'] = data['Items'][0]['Item']['affiliateUrl']
        else:
            item['affiliate_link'] = None
            print(f"Error: 'Items' not found in response for item: {item['title']}")

# ChatGPT APIで商品説明文の生成
def generate_description(items):
    for item in items:
        prompt = f"商品詳細: {item['details']}\n\nこの商品の説明文とPR文を150文字以内で作成してください。"
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "楽天の商品をみんなが買いたくなるように説明してください。"},
                {"role": "user", "content": prompt}
            ]
        )
        item['description'] = completion.choices[0]['message']['content'].strip()

# Twitter投稿関数の追加
async def post_summary(item, image_path):
    try:
        # 画像をアップロードし、メディアIDを取得
        auth = tweepy.OAuth1UserHandler(
            twitter_consumer_key,
            twitter_consumer_secret,
            twitter_access_token,
            twitter_access_token_secret
        )
        api = tweepy.API(auth)
        media = api.media_upload(image_path)
        media_id = media.media_id
        
        # ツイートを投稿
        twitter_client = tweepy.Client(
            bearer_token=twitter_bearer_token,
            consumer_key=twitter_consumer_key,
            consumer_secret=twitter_consumer_secret,
            access_token=twitter_access_token,
            access_token_secret=twitter_access_token_secret
        )
        twitter_client.create_tweet(text=item['description'], media_ids=[media_id])
        print(f"Tweet posted for item: {item['title']}")
        
        # 画像ファイルを削除
        os.remove(image_path)
        print(f"画像ファイル {image_path} を削除しました。")

    except Exception as e:
        print(f"Error during post_summary: {e}")

# Yahooニュースのランキング1位の記事を取得
def get_yahoo_news_top():
    url = 'https://news.yahoo.co.jp/ranking/access/news'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    top_article = soup.select_one('.newsFeed_item_title')
    title = top_article.text.strip()
    link = top_article.parent['href']
    return title, link

# ChatGPT APIでニュース記事の要約を生成
def generate_news_summary(title, link):
    prompt = f"ニュース記事のタイトル: {title}\n記事のURL: {link}\n\nこのニュース記事の要約を150文字以内で作成してください。"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "あなたは、日本のトレンドを知り尽くしているアシスタントです。"},
            {"role": "user", "content": prompt}
        ]
    )
    
    summary = response.choices[0].message['content'].strip()
    return summary

# ニュース要約を投稿
async def post_news_summary():
    try:
        title, link = get_yahoo_news_top()
        summary = generate_news_summary(title, link)
        
        # ツイートを投稿
        twitter_client = tweepy.Client(
            bearer_token=twitter_bearer_token,
            consumer_key=twitter_consumer_key,
            consumer_secret=twitter_consumer_secret,
            access_token=twitter_access_token,
            access_token_secret=twitter_access_token_secret
        )
        twitter_client.create_tweet(text=f"{summary}\n{link}")
        print("News summary tweet posted.")
        
    except Exception as e:
        print(f"Error during post_news_summary: {e}")

# ツイートタスクのスケジューリング
async def tweet_task():
    while True:
        now = datetime.now(pytz.timezone('Asia/Tokyo'))

        items = get_ranking()
        save_images(items)
        get_affiliate_links(items, rakuten_app_id, rakuten_affiliate_id, rakuten_app_secret)
        generate_description(items)
        
        if now.hour == 18 and now.minute == 0:
            print("投稿: 1位の商品")
            await post_summary(items[0], 'images/item_1.jpg')
        elif now.hour == 20 and now.minute == 0:
            print("投稿: 2位の商品")
            await post_summary(items[1], 'images/item_2.jpg')
        elif now.hour == 22 and now.minute == 0:
            print("投稿: 3位の商品")
            await post_summary(items[2], 'images/item_3.jpg')
        
        # ランダムな時間でニュース要約を投稿
        random_times = random.sample([3, 6, 9, 11, 13, 15, 16, 17], 5)
        for hour in random_times:
            if now.hour == hour and now.minute == 15:
                print(f"投稿: Yahooニュースの要約 ({hour}:15)")
                await post_news_summary()

        await asyncio.sleep(900)  # 15分ごとにチェック

# メイン関数
async def main():
    print("Starting the main function")
    print("System Info:", get_system_info())
    task = asyncio.create_task(tweet_task())  # ツイートタスクを追加
    await task

# 実行
if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())


In [1]:


# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

# options = Options()
# options.add_argument('--headless')  # ヘッドレスモードで実行
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# driver = webdriver.Chrome(options=options)
# driver.get('https://www.google.com')
# print("Page title:", driver.title)  # ページタイトルを表示
# driver.quit()



# Pythonファイルの実行
# import json
# import os

# # 設定ファイルのパス
# config_file = 'twitter_key_tokou_use.json'

# # ファイルの存在確認
# if not os.path.exists(config_file):
#     raise FileNotFoundError(f"{config_file} が存在しません。")
# else:
#     print(f"ファイル {config_file} が存在します。")

# # ファイルの内容を読み込み、表示する
# try:
#     with open(config_file, 'r', encoding='utf-8') as f:
#         config = json.load(f)
#         print("Config file loaded successfully:")
#         print(json.dumps(config, indent=4))  # 読み込んだ設定ファイルの内容を表示
# except json.JSONDecodeError as e:
#     raise ValueError(f"JSONファイルの読み込みに失敗しました: {e}")

# # 必要なキーが存在するか確認し、存在する場合は内容を表示
# required_keys = ['twitter', 'rakuten', 'openai']

# all_keys_exist = True

# for key in required_keys:
#     if key in config:
#         print(f"キー '{key}' が {config_file} に存在します。")
#         print(json.dumps(config[key], indent=4))
#     else:
#         print(f"必要なキー '{key}' が {config_file} に存在しません。")
#         all_keys_exist = False

# if not all_keys_exist:
#     raise KeyError("必要なキーが存在しません。")
